In [127]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import requests
import json
from datetime import datetime
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

나이 계산 함수

In [129]:
# 만나이 계산 함수 정의
def calculate_age(birthdate: str) -> str:
    today = datetime.today()  # 오늘 날짜
    birth = datetime.strptime(birthdate, "%Y-%m-%d")
    # 만 나이 계산
    age = today.year - birth.year - ((today.month, today.day) < (birth.month, birth.day))
    return f"{birthdate} 생의 만 나이는 {age}세입니다."

In [130]:
tools = [
    {
        "name": "calculate_age",
        "type": "function",
        "description": "생년월일로 만 나이를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "birthdate": {
                    "type": "string",
                    "description": "생년월일 (YYYY-MM-DD)"
                }
            },
            "required": ["birthdate"]
        }
    }
]

In [131]:
# 사용자 메시지 입력
input_messages = [{"role": "user", "content": "1992-05-01생의 만 나이는?"}]

In [132]:
# 함수 호출하는 기능
response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)

In [133]:
dict(response.output[0])

{'arguments': '{"birthdate":"1992-05-01"}',
 'call_id': 'call_98DFrF0nFGCNHo48v77AS3Lg',
 'name': 'calculate_age',
 'type': 'function_call',
 'id': 'fc_685c9d5033d081998407894cdf9a6a09046f4e10d530dbb0',
 'status': 'completed'}

In [134]:
# 함수 실행
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [136]:
result = calculate_age(**args)
print(result)

1992-05-01 생의 만 나이는 33세입니다.


환율 변환 함수


In [137]:
# 환율 변환 함수 정의
def convert_currency(amount: float, from_currency: str, to_currency: str, rate: float) -> str:
    converted = round(amount * rate, 2)  # 환율 적용 후 소수점 둘째 자리까지 반올림
    return f"{amount} {from_currency}는 환율 {rate} 적용 시 {converted} {to_currency}입니다."

In [138]:
tools = [
    {
        "name": "convert_currency",
        "type": "function",
        "description": "금액과 환율을 입력받아 통화를 변환합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number", "description": "환전할 금액"},
                "from_currency": {"type": "string", "description": "변환 전 통화명"},
                "to_currency": {"type": "string", "description": "변환 후 통화명"},
                "rate": {"type": "number", "description": "환율"}
            },
            "required": ["amount", "from_currency", "to_currency", "rate"]
        }
    }
]

In [139]:
# 사용자 질문 (환율 변환 요청)
input_messages = [{"role": "user", "content": "100달러를 원화로 바꿔줘 (환율 1330원 적용)"}]

In [141]:
response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)

In [143]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [144]:
result = convert_currency(**args)

# 결과 출력
print(result)

100 USD는 환율 1330 적용 시 133000 KRW입니다.


BMI(체질량지수) 계산 함수

In [ ]:
# BMI 계산 함수 정의
def calculate_bmi(height: float, weight: float) -> str:
    height_m = height / 100  # cm → m 변환
    bmi = round(weight / (height_m ** 2), 2)  # BMI 계산, 소수점 둘째 자리 반올림
    return f"키 {height}cm, 몸무게 {weight}kg의 BMI는 {bmi}입니다."

In [146]:
tools = [
    {
        "name": "calculate_bmi",
        "type": "function",
        "description": "키(cm)와 몸무게(kg)로 BMI를 계산합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "height": {"type": "number", "description": "키(cm)"},
                "weight": {"type": "number", "description": "몸무게(kg)"}
            },
            "required": ["height", "weight"]
        }
    }
]

In [147]:
# 사용자 질문 (BMI 계산 요청)
input_messages = [{"role": "user", "content": "키 170cm, 몸무게 65kg의 BMI는?"}]

In [148]:
response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
    tool_choice="auto"
)

In [149]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [150]:
result = calculate_bmi(**args)

# 결과 출력
print(result)

키 170cm, 몸무게 65kg의 BMI는 22.49입니다.
